# Day 1, after lunch: From text files to plots

In this session, we will learn how to read text files and extract relevant information. We will also create plots using the python library ``matplotlib``.

We'll first manually parse a text file in order to understand what needs to happen, before we learn to avoid most of the repetitive code by making use of module functions.

## First example: Weather observations from near Snøheim

Snøheim is a DNT hut close to Snøhetta, Dovrefjell.

In [ ]:
f = open('d1s2/Dovre1-Snoheim.txt', 'r')
lines = f.readlines()
f.close()

In [ ]:
print(lines[0]) # first line of the file
print(lines[-1]) # final line of the file, note the negative indexing

### Parsing
Reading a file with `readlines()` gives us a list of strings, one for each line of the file. In this case each line is composed of a date and time followed by four floating point numbers. 

In order to do anything interesting with the data we need to interpret the strings as dates and numbers.

In [ ]:
from datetime import datetime as dt
import dateutil.parser # powerful datetime string parsing

# store data in lists, one for each column of the file
dates = []
temp = []
ff = []
dd = []
for line in lines:
    line = line.split() # splits a string into substrings by spaces
    datetime_string = ' '.join(line[:2]) # the first two substrings contain time and date
    dates.append(dt.strptime(datetime_string, '%d.%m.%Y %H:%M:%S'))
    # dates.append(dateutil.parser.parse(datetime_string)) # parsing without explicit format string
    temp.append(float(line[3])) # convrts a string into a float
    ff.append(float(line[4]))
    dd.append(float(line[5]))

In [ ]:
print(len(dates), dates[:5])

### Exercise: Read and interpret the tile rro_Bulken.txt
For this exercise you are going to read a file and do some analysis yourself. Start by printing the file to become familiar with its format. Then write a program to answer the following questions:
* On how many days did the runoff exceed 300 m³/s?
* On how many days did the runoff exceed the climatological median flow and the 75-percentile?

Hint: You can use ``sum`` to count the number of ``True`` entries in a list of boolean values.

In [ ]:
f = open('d1s2/rro_Bulken.txt', 'r', encoding='latin1')
lines = f.readlines()
f.close()
dates = []
runoff = []
p50 = []
p75 = []
for line in lines[2:]:
    line = line.split()
    dates.append(dt.strptime(line[0], '%d%m%Y'))
    if line[2] == '----':
        line[2] = 'NaN'
    runoff.append(float(line[2]))
    p50.append(float(line[4]))
    p75.append(float(line[3]))

l = len(runoff)
print('Exceed 300 m3/s:', sum([r > 300 for r in runoff]), 'of', l)
print('Exceed median:', sum([r > m for r, m in zip(runoff, p50)]), 'of', l)
print('Exceed 75p:', sum([r > p for r, p in zip(runoff, p75)]), 'of', l)

This all works fine, but already with this example it became quite repetitive! There must be a simpler solution to this.

## Reading text files using numpy.loadtxt

The name of the function sounds quite promising, so let's give it a naive shot.

In [ ]:
import numpy as np
data = np.loadtxt('d1s2/Dovre1-Snoheim.txt')

That did not work all too well!

The reason is that ``numpy.loadtxt`` tries to convert all values in the matrix to numbers. So we somehow need to convert date strings like the quoted ``'30.06.2012'`` to a number to make this work.

Idea: We could count the number of seconds since a reference date to represent the date as a number. Let's try this.

In [ ]:
DT_REF = dt(1970,1,1)
def parse_time(ibytes):
    ''' Converts a bytes object into a float representing a date and time
    
    We need to return a float instead of a datetime, as the variable type 
    within a numpy array needs to be homogeneous.
    
    To convert the date to a float, we return the number of seconds since 
    a reference time given by DT_REF.
    '''
    
    # Convert bytes to string
    istr = ibytes.decode('latin1')
    # Convert string to datetime
    date = dt.strptime(istr, '%d.%m.%Y %H:%M:%S')
    # Return number of seconds between DT_REF and date
    return (date - DT_REF).total_seconds()

print(parse_time(b'30.06.2012 14:16:00'))

We can then use this function as a converter from string to number. Further, as the date format in the text file contains a space, we'll need to tell ``numpy.loadtxt`` to interpret only tabs as column separators.

In [ ]:
data = np.loadtxt('d1s2/Dovre1-Snoheim.txt', delimiter='\t', converters={
    0: parse_time
})
print(data.shape)
print(data[0])

That works, but but we will then need a way to convert the number back to a ``datetime`` object. Here the ``timedelta`` can help, because we can then just add the given number of seconds on the reference date.

In [ ]:
from datetime import timedelta as td

def float_to_date(val):
    ''' Convert a float back to datetime 
    
    Consistently to ``parse_time'', ``val'' is expected to contain the
    number of seconds since DT_REF. We can then use the timedelta object to 
    add the given number of seconds to DT_REF and return the resulting
    ``datetime'' object.
    '''
    return DT_REF + td(0,val)

print(data[0,0], float_to_date(data[0,0]))

However, manually parsing the time like shown here is quite tedious and error-prone. Fortunately, there is a stable and well-tested version available through ``matplotlib.dates``.

In [ ]:
from matplotlib.dates import strpdate2num, num2date
data = np.loadtxt('d1s2/Dovre1-Snoheim.txt', encoding='utf8', delimiter='\t', converters={
    0: strpdate2num('%d.%m.%Y %H:%M:%S')
})
print(data.shape)
print(data[0,0], num2date(data[0,0]))

## Plotting what we got

First let's extract the data that we want to plot.

In [ ]:
# Create list of dates
dates = [num2date(date) for date in data[:,0]]
# Extract temperature and wind speed, and make them available through easy-to-remember names
airtemp = data[:,2]
windspeed = data[:,3]

Now that we have the data to be plotted ready, let's make our first plot!

In [ ]:
import matplotlib.pyplot as plt

# Some times it's just as easy as this :)
plt.plot(dates, airtemp)

But of course, there's a lot one could criticise about this plot. The dates are quite illegible, and there's no informaiton on what we're actually showing. So let's adapt the plot, and add this info!

In [ ]:
import matplotlib.dates as mdates

plt.plot(dates, airtemp)
plt.ylabel('Temperature [°C]') # Adding labels is easy

# But changing the formatting of the dates is slightly more involved, 
# because the respective functions is only available through the axes object.
ax = plt.gca() # gca: Get Current Axes
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d %b"))

Better! But then let's also focus on a shorter time period, to better see what's actually going on.

In [ ]:
timeslice = slice(800, 3000)
plt.plot(dates[timeslice], airtemp[timeslice])
# has the same effect as
# plt.plot(dates[800:3000], airtemp[800:3000])
# but avoids the repetition.

plt.ylabel('Temperature [°C]')
ax = plt.gca()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d %b"))

Now let's add wind speed measurements in the same plot. We're using the right y-axis for this purpose.

In [ ]:
# 'C0': First color of the standard cycle
plt.plot(dates[timeslice], airtemp[timeslice], color='C0') 
ax1 = plt.gca()
ax1.set_ylabel('Temperature [°C]', color='C0')
# Equivalent to 
#plt.ylabel('Temperature [°C]', color='C0')
# but clearer to read, because the Axes to which we apply the ylabel is explitit

# Change the color of the tick label values
ax1.tick_params(axis='y', labelcolor='C0')

# A new axes object sharing the x-axis with the first
ax2 = ax1.twinx()

# 'C1': Second color of the standard cycle
ax2.plot(dates[timeslice], windspeed[timeslice], color='C1')
ax2.set_ylabel('Wind speed [m/s]', color='C1')
#plt.ylabel('Wind speed [m/s]', color='C1')
ax2.tick_params(axis='y', labelcolor='C1')

# needs to be done after the twinx
ax1.xaxis.set_major_formatter(mdates.DateFormatter("%d %b"))

Looks quite good already, but let's change the Figure size to make the graphs still a bit easier to read. Further, we'll make the graph lines slightly transparent such that the orange line is no longer fully covering the blue one.

In [ ]:
# Defining the figure size 
fig = plt.figure(figsize=(9.6, 3.2), dpi=96)

ax1 = plt.gca()
# alpha=1: fully opaque, alpha=0: fully transparent
ax1.plot(dates[timeslice], airtemp[timeslice], color='C0', alpha=0.7)
ax1.set_ylabel('Temperature [°C]', color='C0') 
ax1.tick_params(axis='y', labelcolor='C0')

ax2 = plt.twinx()
ax2.plot(dates[timeslice], windspeed[timeslice], color='C1', alpha=0.7)
ax2.set_ylabel('Wind speed [m/s]', color='C1')
ax2.tick_params(axis='y', labelcolor='C1')

ax1.xaxis.set_major_formatter(mdates.DateFormatter("%d %b"))

## Exercise: Try to recreate the following plot based on the runoff data from Bulken

 * Use numpy.loadtxt to read the file and parse the dates
 * You might need to define a custom converter to convert ``'----'`` in the text file to NaN values. The built-in function ``float`` might help, also note that ``float('NaN')`` yields a NaN value.
 * The shaded area shows the interval between the 25 and the 75 percentile
 * The functions ``plt.fill_between``, ``plt.semilogy`` and ``plt.legend`` might be helpful 
 * You can use the ``label=value``-keyword argument to most plot commands to provide the labels used in the legend.

In [ ]:
def missing_to_NaN(istr):
    ''' Convert a string containing a number to a float, interpreting unparsible strings as NaN '''
    
    try:
        val = float(istr)
    except ValueError:
        val = float('NaN')
    
    return val

data = np.loadtxt('d1s2/rro_Bulken.txt', encoding='latin1', converters={
    0: strpdate2num('%d%m%Y'),
    1: missing_to_NaN,
    2: missing_to_NaN,
})

Plotting the data.

In [ ]:
dates = [num2date(date) for date in data[:,0]]

fig = plt.figure(figsize=(9.6, 3.2), dpi=96)

plt.semilogy(dates, data[:,4], color='0.4', label='Median') # Median
plt.semilogy(dates, data[:,2], color='C1', label='Observed') # The actual flow
plt.fill_between(dates, data[:,3], data[:,5], color='0.85', label='Interquartile range') # 25 and 75-percentile

plt.ylabel('Runoff at Bulken [m³/s]')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%d %b"))
plt.legend()

plt.savefig('d1s2/ex1.pdf')

### Extension of the exercise: How does that fit with observed precipitation?

Likely, you won't be finished with this exercise by the end of the session. So just take this exercise as far as you get. A solution is provided in the answers-notebook, so you can always come back to it later.

 * Manually parse the input text file. ``numpy.loadtxt`` will not work, because the function does not support footers as present in the file.
 * Plot daily precipitation amount as bars, first into a separate plot, then add it to the one above.
 * Extra challenge: Daily rainfall is defined by MetNorway between 06 UTC the preceeding day to 06 UTC of the given day. Make sure that the precipitation bars are located such along the time axis that the period in which the precipitation is observed matches the time interval covered by the bar.
 
Some hints that might help along the way.

 * You can create bar plots with ``plt.bar``. The width of the bars can be changed by the ``width=value``-keyword argument, which with time date takes a value in the unit of days.
 * You might encounter a mismatch between timezone-aware and timezone-agnostic datetime objects. You can create a datetime object with time zone information using the ``pytz`` module, for example by ``date = dt(2018,9,15,tzinfo=pytz.utc)``.

First step: Reading and parsing the text file.

In [ ]:
# Manual parsing necessary, np.loadtxt does not support footers
f = open('d1s2/rr24_Bulken.txt', encoding='utf8')
lines = f.readlines()
f.close()

dates_start = [] # beginning of the observation intervals
daily_precip = [] # observed values
# Skip 21 header rows and 12 footer rows
for row in lines[21:-12]:
    row = row.split()
    date_given = dt.strptime(row[1], '%d.%m.%Y')
    dates_start.append(date_given - td(0.75))
    daily_precip.append(float(row[2]))

Second step: A quick and dirty plot of the precipitation during this period

In [ ]:
fig = plt.figure(figsize=(9.6, 3.2), dpi=96)
plt.bar(dates_start, daily_precip, width=0.8)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%d %b"))
plt.savefig('d1s2/ex2_step1.pdf')

Side remark: Comparing datetime object with/without time zone information.

In [ ]:
# dates, created through num2date, does contain time zone information, 
# where as those in dates_start, created through strptime, do not
print(dates[0], dates_start[0])

In [ ]:
# Will yield an error, because those two types of datetime are not comparable
dates[0] > dates_start[0]

Third and final step: Putting everything together.

In [ ]:
# Module for dealing with time zones
import pytz

# Then combining everything together, considering the period 15 September - 31 October
mask_precip = np.array([date >= dt(2018,9,15) and date < dt(2018,11,1) for date in dates_start], dtype='bool')
mask_runoff = np.array([date >= dt(2018,9,15,tzinfo=pytz.utc) and date < dt(2018,11,1,tzinfo=pytz.utc) for date in dates], dtype='bool')
dates = np.array(dates)
dates_start = np.array(dates_start)
daily_precip = np.array(daily_precip)

fig = plt.figure(figsize=(9.6, 4.8), dpi=96)

plt.semilogy(dates[mask_runoff], data[mask_runoff,4], color='0.4', label='Median') # Median
plt.semilogy(dates[mask_runoff], data[mask_runoff,2], color='C1', label='Observed') # The actual flow
plt.fill_between(dates[mask_runoff], data[mask_runoff,3], data[mask_runoff,5], color='0.85', label='Interquartile range') # 25 and 75-percentile
plt.ylabel('Runoff at Bulken [m³/s]')
plt.legend(loc='upper right')

ax2 = plt.twinx()
ax2.bar(dates_start[mask_precip], daily_precip[mask_precip], width=0.9, alpha=0.5, color='C0')
ax2.set_ylabel('Daily precip. at Bulken [mm]', color='C0')
ax2.tick_params(axis='y', labelcolor='C0')

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%d %b"))
plt.savefig('d1s2/ex2.pdf')